# Notes

Clean filtered data. Fields can easily be copied to top of analysis notebooks.


In [1]:
import pandas as pd
train_identity = pd.read_csv('Data/train_identity.csv',index_col='TransactionID')
train_transaction = pd.read_csv('Data/train_transaction.csv',index_col='TransactionID')
train_transaction = train_transaction.merge(train_identity, how='left', left_index=True, right_index=True)

In [5]:
# filter columns using threshold of maximum 20% NAs in column
threshold = 0.2
train_transaction_selection = pd.DataFrame()
for colName in train_transaction:
    naCountColumn = train_transaction[colName].isna().sum()/len(train_transaction)
    if (naCountColumn < threshold):
        train_transaction_selection[colName] = train_transaction[colName]
train_transaction = train_transaction_selection

In [39]:
# !! Choose policy for filling NAs. Option 1 uses mode, Option 2 uses -999


# Option 1
for colName in train_transaction_selection:
    train_transaction_selection.loc[:, (colName)].fillna(train_transaction_selection[colName].mode().iloc[0], inplace = True)


# Option 2
train_transaction.fillna(-999, inplace=True)

